<a href="https://colab.research.google.com/github/pratikabhang/AI-Driven-Cybersecurity-Threat-Prediction-Platform/blob/main/sql_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Task 0: Setup - create or recreate SQLite DB and helpers
import sqlite3, os, textwrap

DB_FILE = "CyberSecurityDB.db"
# Remove old DB to ensure a clean start (comment out to keep data between runs)
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)

conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

def run_select(sql, params=None):
    cursor.execute(sql) if params is None else cursor.execute(sql, params)
    rows = cursor.fetchall()
    if cursor.description:
        cols = [d[0] for d in cursor.description]
        print(" | ".join(cols))
        print("-" * max(20, len(" | ".join(cols))))
    for r in rows:
        print(r)
    return rows

def run_exec(sql, params=None):
    cursor.execute(sql) if params is None else cursor.execute(sql, params)
    conn.commit()
    print("Executed:", sql.strip().splitlines()[0])

print("Connected to", DB_FILE)


Connected to CyberSecurityDB.db


In [16]:
# Task 1: Create a Database (SQLite uses a file - already created by connect)
print("Database file (SQLite) created/opened:", DB_FILE)


Database file (SQLite) created/opened: CyberSecurityDB.db


In [17]:
# Task 2: Use the Database (not required in SQLite; connection is active)
print("Using database:", DB_FILE)


Using database: CyberSecurityDB.db


In [18]:
# Task 3: Create Users table
run_exec("""
CREATE TABLE IF NOT EXISTS Users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    city TEXT
);
""")


Executed: CREATE TABLE IF NOT EXISTS Users (


In [19]:
# Task 4: Insert sample Users data (INSERT OR REPLACE to avoid duplicates)
users_data = [
    (1, 'Alex', 'alex@mail.com', 'New York'),
    (2, 'Priya', 'priya@mail.com', 'London'),
    (3, 'Chen', 'chen@mail.com', 'Paris'),
    (4, 'Fatima', 'fatima@mail.com', 'Berlin'),
    (5, 'Ravi', 'ravi@mail.com', 'New York'),
    (6, 'Asha', 'asha@mail.com', 'London')
]
cursor.executemany("""
INSERT OR REPLACE INTO Users (id, name, email, city) VALUES (?, ?, ?, ?);
""", users_data)
conn.commit()
print("Inserted/replaced sample Users data.")


Inserted/replaced sample Users data.


In [20]:
# Task 5: Retrieve All Data
run_select("SELECT * FROM Users;")


id | name | email | city
------------------------
(1, 'Alex', 'alex@mail.com', 'New York')
(2, 'Priya', 'priya@mail.com', 'London')
(3, 'Chen', 'chen@mail.com', 'Paris')
(4, 'Fatima', 'fatima@mail.com', 'Berlin')
(5, 'Ravi', 'ravi@mail.com', 'New York')
(6, 'Asha', 'asha@mail.com', 'London')


[(1, 'Alex', 'alex@mail.com', 'New York'),
 (2, 'Priya', 'priya@mail.com', 'London'),
 (3, 'Chen', 'chen@mail.com', 'Paris'),
 (4, 'Fatima', 'fatima@mail.com', 'Berlin'),
 (5, 'Ravi', 'ravi@mail.com', 'New York'),
 (6, 'Asha', 'asha@mail.com', 'London')]

In [21]:
# Task 6: Retrieve name, city
run_select("SELECT name, city FROM Users;")


name | city
--------------------
('Alex', 'New York')
('Priya', 'London')
('Chen', 'Paris')
('Fatima', 'Berlin')
('Ravi', 'New York')
('Asha', 'London')


[('Alex', 'New York'),
 ('Priya', 'London'),
 ('Chen', 'Paris'),
 ('Fatima', 'Berlin'),
 ('Ravi', 'New York'),
 ('Asha', 'London')]

In [22]:
# Task 7: WHERE city = 'New York'
run_select("SELECT * FROM Users WHERE city = 'New York';")


id | name | email | city
------------------------
(1, 'Alex', 'alex@mail.com', 'New York')
(5, 'Ravi', 'ravi@mail.com', 'New York')


[(1, 'Alex', 'alex@mail.com', 'New York'),
 (5, 'Ravi', 'ravi@mail.com', 'New York')]

In [23]:
# Task 8: OR operator example (New York or London)
run_select("SELECT * FROM Users WHERE city = 'New York' OR city = 'London';")


id | name | email | city
------------------------
(1, 'Alex', 'alex@mail.com', 'New York')
(2, 'Priya', 'priya@mail.com', 'London')
(5, 'Ravi', 'ravi@mail.com', 'New York')
(6, 'Asha', 'asha@mail.com', 'London')


[(1, 'Alex', 'alex@mail.com', 'New York'),
 (2, 'Priya', 'priya@mail.com', 'London'),
 (5, 'Ravi', 'ravi@mail.com', 'New York'),
 (6, 'Asha', 'asha@mail.com', 'London')]

In [24]:
# Task 9: ORDER BY name ASC
run_select("SELECT * FROM Users ORDER BY name ASC;")


id | name | email | city
------------------------
(1, 'Alex', 'alex@mail.com', 'New York')
(6, 'Asha', 'asha@mail.com', 'London')
(3, 'Chen', 'chen@mail.com', 'Paris')
(4, 'Fatima', 'fatima@mail.com', 'Berlin')
(2, 'Priya', 'priya@mail.com', 'London')
(5, 'Ravi', 'ravi@mail.com', 'New York')


[(1, 'Alex', 'alex@mail.com', 'New York'),
 (6, 'Asha', 'asha@mail.com', 'London'),
 (3, 'Chen', 'chen@mail.com', 'Paris'),
 (4, 'Fatima', 'fatima@mail.com', 'Berlin'),
 (2, 'Priya', 'priya@mail.com', 'London'),
 (5, 'Ravi', 'ravi@mail.com', 'New York')]

In [25]:
# Task 10: DISTINCT cities
run_select("SELECT DISTINCT city FROM Users;")


city
--------------------
('New York',)
('London',)
('Paris',)
('Berlin',)


[('New York',), ('London',), ('Paris',), ('Berlin',)]

In [26]:
# Task 11: Update id=1 city to Boston
run_exec("UPDATE Users SET city = 'Boston' WHERE id = 1;")
run_select("SELECT * FROM Users WHERE id = 1;")


Executed: UPDATE Users SET city = 'Boston' WHERE id = 1;
id | name | email | city
------------------------
(1, 'Alex', 'alex@mail.com', 'Boston')


[(1, 'Alex', 'alex@mail.com', 'Boston')]

In [27]:
# Task 12: Delete user with id = 6 (if exists)
run_exec("DELETE FROM Users WHERE id = 6;")
run_select("SELECT * FROM Users;")
# Reinsert id 6 to keep sample data complete
run_exec("INSERT OR REPLACE INTO Users (id, name, email, city) VALUES (6, 'Asha', 'asha@mail.com', 'London');")


Executed: DELETE FROM Users WHERE id = 6;
id | name | email | city
------------------------
(1, 'Alex', 'alex@mail.com', 'Boston')
(2, 'Priya', 'priya@mail.com', 'London')
(3, 'Chen', 'chen@mail.com', 'Paris')
(4, 'Fatima', 'fatima@mail.com', 'Berlin')
(5, 'Ravi', 'ravi@mail.com', 'New York')
Executed: INSERT OR REPLACE INTO Users (id, name, email, city) VALUES (6, 'Asha', 'asha@mail.com', 'London');


In [28]:
# Task 13: Add column age (if not exists) and set ages
# SQLite ignores ADD COLUMN if column exists, so guard:
try:
    run_exec("ALTER TABLE Users ADD COLUMN age INTEGER;")
except Exception as e:
    print("ALTER TABLE ADD COLUMN skipped or not supported:", e)

age_updates = [(30,1),(28,2),(35,3),(26,4),(32,5),(27,6)]
cursor.executemany("UPDATE Users SET age = ? WHERE id = ?;", age_updates)
conn.commit()
run_select("SELECT id, name, city, age FROM Users;")


Executed: ALTER TABLE Users ADD COLUMN age INTEGER;
id | name | city | age
----------------------
(1, 'Alex', 'Boston', 30)
(2, 'Priya', 'London', 28)
(3, 'Chen', 'Paris', 35)
(4, 'Fatima', 'Berlin', 26)
(5, 'Ravi', 'New York', 32)
(6, 'Asha', 'London', 27)


[(1, 'Alex', 'Boston', 30),
 (2, 'Priya', 'London', 28),
 (3, 'Chen', 'Paris', 35),
 (4, 'Fatima', 'Berlin', 26),
 (5, 'Ravi', 'New York', 32),
 (6, 'Asha', 'London', 27)]

In [29]:
# Task 14: Delete a Column - SQLite has no direct DROP COLUMN.
# Helper is provided but NOT executed automatically to avoid accidental data loss.
print("SQLite cannot DROP COLUMN directly. If you really want to drop 'age', call the helper below.")

def sqlite_drop_column(table_name, drop_column):
    cursor.execute(f"PRAGMA table_info({table_name});")
    cols = cursor.fetchall()
    col_names = [c[1] for c in cols if c[1] != drop_column]
    col_defs = []
    for c in cols:
        if c[1] == drop_column: continue
        col_defs.append(f"{c[1]} {c[2]}")
    new_table = table_name + "_new"
    cursor.execute(f"CREATE TABLE {new_table} ({', '.join(col_defs)});")
    cursor.execute(f"INSERT INTO {new_table} ({', '.join(col_names)}) SELECT {', '.join(col_names)} FROM {table_name};")
    cursor.execute(f"DROP TABLE {table_name};")
    cursor.execute(f"ALTER TABLE {new_table} RENAME TO {table_name};")
    conn.commit()
    print(f"Dropped column '{drop_column}' from {table_name}")

# Example (commented): sqlite_drop_column('Users','age')


SQLite cannot DROP COLUMN directly. If you really want to drop 'age', call the helper below.


In [30]:
# Task 15: Rename column city -> location
# Newer SQLite supports RENAME COLUMN
try:
    run_exec("ALTER TABLE Users RENAME COLUMN city TO location;")
except Exception as e:
    print("Rename column failed (older SQLite):", e)
run_select("PRAGMA table_info(Users);")


Executed: ALTER TABLE Users RENAME COLUMN city TO location;
cid | name | type | notnull | dflt_value | pk
---------------------------------------------
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 0, None, 0)
(2, 'email', 'TEXT', 0, None, 0)
(3, 'location', 'TEXT', 0, None, 0)
(4, 'age', 'INTEGER', 0, None, 0)


[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'email', 'TEXT', 0, None, 0),
 (3, 'location', 'TEXT', 0, None, 0),
 (4, 'age', 'INTEGER', 0, None, 0)]

In [31]:
# Task 16: MAX(age)
run_select("SELECT MAX(age) AS MaxAge FROM Users;")


MaxAge
--------------------
(35,)


[(35,)]

In [32]:
# Task 17: MIN(age)
run_select("SELECT MIN(age) AS MinAge FROM Users;")


MinAge
--------------------
(26,)


[(26,)]

In [33]:
# Task 18: AVG(age)
run_select("SELECT AVG(age) AS AvgAge FROM Users;")


AvgAge
--------------------
(29.666666666666668,)


[(29.666666666666668,)]

In [34]:
# Task 19: COUNT(*) total users
run_select("SELECT COUNT(*) AS TotalUsers FROM Users;")


TotalUsers
--------------------
(6,)


[(6,)]

In [35]:
# Task 20: GROUP BY location
run_select("""
SELECT location, COUNT(*) AS NumUsers
FROM Users
GROUP BY location;
""")


location | NumUsers
--------------------
('Berlin', 1)
('Boston', 1)
('London', 2)
('New York', 1)
('Paris', 1)


[('Berlin', 1), ('Boston', 1), ('London', 2), ('New York', 1), ('Paris', 1)]

In [36]:
# Task 21: HAVING (locations with more than 1 user)
run_select("""
SELECT location, COUNT(*) AS NumUsers
FROM Users
GROUP BY location
HAVING COUNT(*) > 1;
""")


location | NumUsers
--------------------
('London', 2)


[('London', 2)]

In [37]:
# Task 22: LIKE operator - names starting with 'A'
run_select("SELECT * FROM Users WHERE name LIKE 'A%';")


id | name | email | location | age
----------------------------------
(1, 'Alex', 'alex@mail.com', 'Boston', 30)
(6, 'Asha', 'asha@mail.com', 'London', 27)


[(1, 'Alex', 'alex@mail.com', 'Boston', 30),
 (6, 'Asha', 'asha@mail.com', 'London', 27)]

In [38]:
# Task 23: IN operator
run_select("SELECT * FROM Users WHERE location IN ('London','Paris','Berlin');")


id | name | email | location | age
----------------------------------
(2, 'Priya', 'priya@mail.com', 'London', 28)
(3, 'Chen', 'chen@mail.com', 'Paris', 35)
(4, 'Fatima', 'fatima@mail.com', 'Berlin', 26)
(6, 'Asha', 'asha@mail.com', 'London', 27)


[(2, 'Priya', 'priya@mail.com', 'London', 28),
 (3, 'Chen', 'chen@mail.com', 'Paris', 35),
 (4, 'Fatima', 'fatima@mail.com', 'Berlin', 26),
 (6, 'Asha', 'asha@mail.com', 'London', 27)]

In [39]:
# Task 24: BETWEEN operator (age between 28 and 33)
run_select("SELECT * FROM Users WHERE age BETWEEN 28 AND 33;")


id | name | email | location | age
----------------------------------
(1, 'Alex', 'alex@mail.com', 'Boston', 30)
(2, 'Priya', 'priya@mail.com', 'London', 28)
(5, 'Ravi', 'ravi@mail.com', 'New York', 32)


[(1, 'Alex', 'alex@mail.com', 'Boston', 30),
 (2, 'Priya', 'priya@mail.com', 'London', 28),
 (5, 'Ravi', 'ravi@mail.com', 'New York', 32)]

In [40]:
# Task 25: Create Orders table
run_exec("""
CREATE TABLE IF NOT EXISTS Orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    amount REAL,
    FOREIGN KEY (user_id) REFERENCES Users(id)
);
""")


Executed: CREATE TABLE IF NOT EXISTS Orders (


In [41]:
# Task 26: Insert sample Orders (INSERT OR REPLACE to avoid duplicates)
orders_data = [
    (1, 1, 120.50),
    (2, 2, 750.00),
    (3, 3, 620.75),
    (4, 1, 45.00),
    (5, 5, 980.00),
    (6, 2, 15.00),
    (7, 4, 510.00),
    (8, 5, 2000.00)
]
cursor.executemany("INSERT OR REPLACE INTO Orders (order_id, user_id, amount) VALUES (?, ?, ?);", orders_data)
conn.commit()
print("Inserted/replaced sample Orders data.")
run_select("SELECT * FROM Orders;")


Inserted/replaced sample Orders data.
order_id | user_id | amount
---------------------------
(1, 1, 120.5)
(2, 2, 750.0)
(3, 3, 620.75)
(4, 1, 45.0)
(5, 5, 980.0)
(6, 2, 15.0)
(7, 4, 510.0)
(8, 5, 2000.0)


[(1, 1, 120.5),
 (2, 2, 750.0),
 (3, 3, 620.75),
 (4, 1, 45.0),
 (5, 5, 980.0),
 (6, 2, 15.0),
 (7, 4, 510.0),
 (8, 5, 2000.0)]

In [42]:
# Task 27: INNER JOIN - show user names with order amounts
run_select("""
SELECT Users.name, Orders.amount
FROM Users
INNER JOIN Orders ON Users.id = Orders.user_id
ORDER BY Orders.amount DESC;
""")


name | amount
--------------------
('Ravi', 2000.0)
('Ravi', 980.0)
('Priya', 750.0)
('Chen', 620.75)
('Fatima', 510.0)
('Alex', 120.5)
('Alex', 45.0)
('Priya', 15.0)


[('Ravi', 2000.0),
 ('Ravi', 980.0),
 ('Priya', 750.0),
 ('Chen', 620.75),
 ('Fatima', 510.0),
 ('Alex', 120.5),
 ('Alex', 45.0),
 ('Priya', 15.0)]

In [43]:
# Task 28: LEFT JOIN - users + their orders (NULL for no orders)
run_select("""
SELECT Users.name, Orders.amount
FROM Users
LEFT JOIN Orders ON Users.id = Orders.user_id
ORDER BY Users.id;
""")


name | amount
--------------------
('Alex', 45.0)
('Alex', 120.5)
('Priya', 15.0)
('Priya', 750.0)
('Chen', 620.75)
('Fatima', 510.0)
('Ravi', 980.0)
('Ravi', 2000.0)
('Asha', None)


[('Alex', 45.0),
 ('Alex', 120.5),
 ('Priya', 15.0),
 ('Priya', 750.0),
 ('Chen', 620.75),
 ('Fatima', 510.0),
 ('Ravi', 980.0),
 ('Ravi', 2000.0),
 ('Asha', None)]

In [44]:
# Task 29: RIGHT JOIN alternative via Orders LEFT JOIN Users
run_select("""
SELECT Users.name, Orders.amount
FROM Orders
LEFT JOIN Users ON Users.id = Orders.user_id
ORDER BY Orders.order_id;
""")


name | amount
--------------------
('Alex', 120.5)
('Priya', 750.0)
('Chen', 620.75)
('Alex', 45.0)
('Ravi', 980.0)
('Priya', 15.0)
('Fatima', 510.0)
('Ravi', 2000.0)


[('Alex', 120.5),
 ('Priya', 750.0),
 ('Chen', 620.75),
 ('Alex', 45.0),
 ('Ravi', 980.0),
 ('Priya', 15.0),
 ('Fatima', 510.0),
 ('Ravi', 2000.0)]

In [45]:
# Task 30: Subquery - users who have any order > 500
run_select("""
SELECT name
FROM Users
WHERE id IN (SELECT user_id FROM Orders WHERE amount > 500);
""")


name
--------------------
('Priya',)
('Chen',)
('Fatima',)
('Ravi',)


[('Priya',), ('Chen',), ('Fatima',), ('Ravi',)]

In [46]:
# Task 31: Create view HighValueOrders (orders > 500) and select from it
run_exec("""
CREATE VIEW IF NOT EXISTS HighValueOrders AS
SELECT Users.name, Orders.amount
FROM Users
JOIN Orders ON Users.id = Orders.user_id
WHERE Orders.amount > 500;
""")
run_select("SELECT * FROM HighValueOrders;")


Executed: CREATE VIEW IF NOT EXISTS HighValueOrders AS
name | amount
--------------------
('Priya', 750.0)
('Chen', 620.75)
('Ravi', 980.0)
('Fatima', 510.0)
('Ravi', 2000.0)


[('Priya', 750.0),
 ('Chen', 620.75),
 ('Ravi', 980.0),
 ('Fatima', 510.0),
 ('Ravi', 2000.0)]

In [47]:
# Task 32: Drop Orders table and the view
run_exec("DROP TABLE IF EXISTS Orders;")
run_exec("DROP VIEW IF EXISTS HighValueOrders;")
# Show final Users table
run_select("SELECT * FROM Users;")


Executed: DROP TABLE IF EXISTS Orders;
Executed: DROP VIEW IF EXISTS HighValueOrders;
id | name | email | location | age
----------------------------------
(1, 'Alex', 'alex@mail.com', 'Boston', 30)
(2, 'Priya', 'priya@mail.com', 'London', 28)
(3, 'Chen', 'chen@mail.com', 'Paris', 35)
(4, 'Fatima', 'fatima@mail.com', 'Berlin', 26)
(5, 'Ravi', 'ravi@mail.com', 'New York', 32)
(6, 'Asha', 'asha@mail.com', 'London', 27)


[(1, 'Alex', 'alex@mail.com', 'Boston', 30),
 (2, 'Priya', 'priya@mail.com', 'London', 28),
 (3, 'Chen', 'chen@mail.com', 'Paris', 35),
 (4, 'Fatima', 'fatima@mail.com', 'Berlin', 26),
 (5, 'Ravi', 'ravi@mail.com', 'New York', 32),
 (6, 'Asha', 'asha@mail.com', 'London', 27)]